# **CATS VS DOGS** | IMAGE CLASSIFIER | AUTOKERAS

In [ ]:
from autokeras.image.image_supervised import ImageClassifier
from autokeras.image.image_supervised import load_image_dataset

from os import listdir
from os.path import isfile, join

import torch
from keras.models import load_model

### **1. LOAD DATA**

In [ ]:
# Show path to images
train_path = './data/dogsCats128rgb_train'
train_labels = './data/dogsCats_train.csv'
test_path = './data/dogsCats128rgb_val'
test_labels = './data/dogsCats_val.csv'

In [ ]:
# Read data
X_train, Y_train = load_image_dataset(csv_file_path=train_labels, images_path=train_path)
X_train, Y_test = load_image_dataset(csv_file_path=test_labels, images_path=test_path)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

### **2. SEARCH FOR BEST MODEL**

In [ ]:
# make instance of the model
clf = ImageClassifier(verbose = True, searcher_args = {'trainer_args':{'max_iter_num': 25}})

# train the model
clf.fit(X_train, Y_train, time_limit = 4 * 60 * 60)

### **3. FIT BEST MODEL**

In [ ]:
# final training
clf.final_fit(X_train, Y_train, X_test, Y_test, retrain = True, trainer_args={'max_iter_num':10})

### **4. EVALUATE THE MODEL**

In [ ]:
clf.evaluate(X_test, Y_test)

### **5. BEST MODEL**

In [ ]:
# load_best_model() finds the best model based on the metric of highest accuracy
best = clf.load_searcher().load_best_model()

best_torchModel = best.produce_model()                       # best PyTorch model
best_kerasModel = best.produce_keras_model()                 # best Keras model

In [ ]:
# display number of layers of the best model
best.n_layers

In [ ]:
# display architecture of the best PyTorch Model
best_torchModel

In [ ]:
# architecture of the best Keras model
best_kerasModel

### **6. SAVE MODELS**

In [ ]:
# Save PyTorch model
torch.save(best.produce_model(),'dogsAndCats_model.pt')
loadedTorchModel = torch.load('dogsAndCats_model.pt')

# Save Keras model
best_kerasModel.save('dogsAndCats_model.h5')
loadedKerasModel = load_model('dogsAndCats_model.h5')

### **7. SANITY CHECK**

In [ ]:
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import copy


# Transform Images
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
class_names = ['dog','cat']

# switch to CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# switch to GPU if available
loadedTorchModel = loadedTorchModel.to(device)

# visualize model predictions
visualize_model(loadedTorchModel,8)